In [1]:
from google.colab import drive

In [2]:
drive.mount("/content/drive/")

Mounted at /content/drive/


In [3]:
cd/content/drive/MyDrive/ML challenge

/content/drive/MyDrive/ML challenge


In [4]:
from transformers import pipeline
import pandas as pd
import re
from tqdm import tqdm
from time import time

In [5]:
qa_pipeline = pipeline('question-answering', model="deepset/roberta-base-squad2-distilled", device='cuda')
# qa_pipeline = pipeline('question-answering', model="deepset/roberta-base-squad2", device=0)
context = ""
question = "Answer the numerical value with the unit. What is the %% mentioned in the description?"
voltage_question = "Answer the numerical value with the unit. What is the voltage range mentioned in the description?"
depth_question = "Answer the largest numerical value with the unit"

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/729 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/496M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/295 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [6]:
def process_text(answer, entity):
    # Clean up the output of the LLM
    entity_unit_map = {
        "width": {"centimetre", "foot", "millimetre", "metre", "inch", "yard"},
        "depth": {"centimetre", "foot", "millimetre", "metre", "inch", "yard"},
        "height": {"centimetre", "foot", "millimetre", "metre", "inch", "yard"},
        "item_weight": {
            "milligram",
            "kilogram",
            "microgram",
            "gram",
            "ounce",
            "ton",
            "pound",
        },
        "voltage": {"millivolt", "kilovolt", "volt"},
        "wattage": {"kilowatt", "watt"},
        "item_volume": {
            "cubic foot",
            "microlitre",
            "cup",
            "fluid ounce",
            "centilitre",
            "imperial gallon",
            "pint",
            "decilitre",
            "litre",
            "millilitre",
            "quart",
            "cubic inch",
            "gallon",
            "ounce",
        },
        "maximum_weight_recommendation": {
            "milligram",
            "kilogram",
            "microgram",
            "gram",
            "ounce",
            "ton",
            "pound",
        },
    }
    unit_short_form = {
        "cm": "centimetre",
        "mm": "millimetre",
        "m": "metre",
        "in": "inch",
        "ft": "foot",
        "yd": "yard",
        "g": "gram",
        "kg": "kilogram",
        "mg": "milligram",
        "lb": "pound",
        "oz": "ounce",
        "ton": "ton",
        "ug": "microgram",
        "lbs": "pound",
        "9": "gram",
        "ozs": "ounce",
        "mv": "millivolt",
        "kv": "kilovolt",
        "v": "volt",
        "kw": "kilowatt",
        "w": "watt",
        "cf": "cubic foot",
        "ul": "microlitre",
        "fl oz": "fluid ounce",
        "cl": "centilitre",
        "gal": "imperial gallon",
        "pt": "pint",
        "dl": "decilitre",
        "l": "litre",
        "ml": "millilitre",
        "qt": "quart",
        "cu in": "cubic inch",
        "gals": "gallon",
        "c in": "cubic inch",
        "cu ft": "cubic foot",
        "o2": "ounce",
        "0z": "ounce",
        "k9": "kilogram",
        "\'\'": "inch",
        "``": "inch",
        "\"": "inch",
        # If no unit is mentioned, assume it is inches
        "": "inch",
        "c": "centimetre",
    }
    # Replace common OCR errors
    answer = answer.replace(",", ".")

    if entity == "voltage":
    # Answer should be of the form [x, y] volt
    # Extract the voltage range
        pattern = r"[0-9IJOQSZDLBLBLBLBLBLBLB]+(\.[0-9IJOQSZDLB]+)?[-/][0-9IJOQSZDLB]+(\.[0-9IJOQSZDLB]+)?\s*(mv|kv|v|milli|kilo)|[0-9IJOQSZDLB]+\s*(mv|kv|v|milli|kilo)"
        voltage_ranges = re.finditer(pattern, answer, re.IGNORECASE)
        if not voltage_ranges:
            return None
        for vr in voltage_ranges:
            voltage_range = vr.group(0)
            # Extract the unit
            unit = re.search(
                r"\s*(mv|kv|v|milli|kilo)", voltage_range, re.IGNORECASE
            )
            if not unit:
                continue
            unit = unit.group(0).strip()
            voltage_range = voltage_range.replace(unit, "").strip()
            voltage_range = (
                voltage_range.replace("I", "1")
                .replace("i", "1")
                .replace("J", "1")
                .replace("j", "1")
                .replace("O", "0")
                .replace("o", "0")
                .replace("D", "0")
            )
            # Convert the unit to full form
            unit = unit.lower()
            if unit in unit_short_form:
                unit = unit_short_form[unit]
            # Check if the unit is valid
            if unit in entity_unit_map[entity]:
                if "-" in voltage_range or "/" in voltage_range:
                    voltage_range = voltage_range.split("-") if "-" in voltage_range else voltage_range.split("/")
                    voltage_range = float(voltage_range[0])
                    return str(voltage_range) + " " + unit




    entity_unit = entity_unit_map[entity]
    regex_map = {
        "item_weight": r"[0-9IJOQSZDLB]+(\.[0-9IJOQSZDLB]+)?\s*(g|kg|mg|ug|oz|ton|lb|lbs|ozs|o2|0z|k9|kilo|milli|micro|ounce|tonne|pound|gram)",
        "width": r"[0-9IJOQSZDLB]+(\.[0-9IJOQSZDLB]+)?\s*(c|cm|mm|m|in|ft|yd|centi|milli|inch|foot|yard|\'\'|``|\"|feet)",
        "depth": r"[0-9IJOQSZDLB]+(\.[0-9IJOQSZDLB]+)?\s*(c|cm|mm|m|in|ft|yd|centi|milli|inch|foot|yard|\'\'|``|\"|feet)",
        "height": r"[0-9IJOQSZDLB]+(\.[0-9IJOQSZDLB]+)?\s*(c|cm|mm|m|in|ft|yd|centi|milli|inch|foot|yard|\'\'|``|\"|feet)",
        "voltage": r"[0-9IJOQSZDLB]+(\.[0-9IJOQSZDLB]+)?\s*(mv|kv|v|milli|kilo)",
        "wattage": r"[0-9IJOQSZDLB]+(\.[0-9IJOQSZDLB]+)?\s*(kw|w|kilo)",
        "item_volume": r"[0-9IJOQSZDLB]+(\.[0-9IJOQSZDLB]+)?\s*(cf|ul|fl oz|cl|gal|pt|dl|l|ml|qt|cu in|gals|c in|cu ft|o2|oz|ozs|0z|cubic|micro|cup|fluid|centi|imperial|pint|decilitre|litre|millilitre|quart|gallon|ounce)",
        "maximum_weight_recommendation": r"[0-9IJOQSZDLB]+(\.[0-9IJOQSZDLB]+)?\s*(g|kg|mg|ug|oz|ton|lb|lbs|ozs|o2|02|0z|k9|kilo|milli|micro|ounce|tonne|pound|gram)",
    }
    reverse_regex_map = {
        "item_weight": r"\s*(g|kg|mg|ug|oz|ton|lb|lbs|ozs|o2|0z|k9|kilo|milli|micro|ounce|tonne|pound|gram)\s*[0-9IJOQSZDLB]+(\.[0-9IJOQSZDLB]+)?",
        "width": r"\s*(c|cm|mm|m|in|ft|yd|centi|milli|inch|foot|yard|\'\'|``|\"|feet)\s*[0-9IJOQSZDLB]+(\.[0-9IJOQSZDLB]+)?",
        "depth": r"\s*(c|cm|mm|m|in|ft|yd|centi|milli|inch|foot|yard|\'\'|``|\"|feet)\s*[0-9IJOQSZDLB]+(\.[0-9IJOQSZDLB]+)?",
        "height": r"\s*(c|cm|mm|m|in|ft|yd|centi|milli|inch|foot|yard|\'\'|``|\"|feet)\s*[0-9IJOQSZDLB]+(\.[0-9IJOQSZDLB]+)?",
        "voltage": r"\s*(mv|kv|v|milli|kilo)\s*[0-9IJOQSZDLB]+(\.[0-9IJOQSZDLB]+)?",
        "wattage": r"\s*(kw|w|kilo)\s*[0-9IJOQSZDLB]+(\.[0-9IJOQSZDLB]+)?",
        "item_volume": r"\s*(cf|ul|fl oz|cl|gal|pt|dl|l|ml|qt|cu in|gals|c in|cu ft|o2|oz|ozs|0z|cubic|micro|cup|fluid|centi|imperial|pint|decilitre|litre|millilitre|quart|gallon|ounce)\s*[0-9IJOQSZDLB]+(\.[0-9IJOQSZDLB]+)?",
        "maximum_weight_recommendation": r"\s*(g|kg|mg|ug|oz|ton|lb|lbs|ozs|o2|02|0z|k9|kilo|milli|micro|ounce|tonne|pound|gram)\s*[0-9IJOQSZDLB]+(\.[0-9IJOQSZDLB]+)?",
    }
    regex_units = {
        "item_weight": r"\s*(g|kg|mg|ug|oz|ton|lb|lbs|ozs|o2|0z|k9|kilo|milli|micro|ounce|tonne|pound|gram)",
        "width": r"\s*(c|cm|mm|m|in|ft|yd|centi|milli|inch|foot|yard|\'\'|``|\"|feet)",
        "depth": r"\s*(c|cm|mm|m|in|ft|yd|centi|milli|inch|foot|yard|\'\'|``|\"|feet)",
        "height": r"\s*(c|cm|mm|m|in|ft|yd|centi|milli|inch|foot|yard|\'\'|``|\"|feet)",
        "voltage": r"\s*(mv|kv|v|milli|kilo)",
        "wattage": r"\s*(kw|w|kilo)",
        "item_volume": r"\s*(cf|ul|fl oz|cl|gal|pt|dl|l|ml|qt|cu in|gals|c in|cu ft|o2|oz|ozs|0z|cubic|micro|cup|fluid|centi|imperial|pint|decilitre|litre|millilitre|quart|gallon|ounce)",
        "maximum_weight_recommendation": r"\s*(g|kg|mg|ug|oz|ton|lb|lbs|ozs|o2|0z|k9|kilo|milli|micro|ounce|tonne|pound|gram)",
    }
    # Remove igh which will cause false positives
    answer = answer.replace("igh", "")
    # Remove ima which will cause false positives
    answer = answer.replace("ima", "")
    # Replace comma with dot
    answer = answer.replace(",", ".")
    # Match the regex ignoring case
    match = re.finditer(regex_map[entity], answer, re.IGNORECASE)
    reverse_match = re.finditer(reverse_regex_map[entity], answer, re.IGNORECASE)

    # Process each match
    for m in match:
        # Extract the number and the unit
        # Extract the matched value
        value = m.group(0)
        # Extract the unit
        unit = re.search(
            regex_units[entity], value, re.IGNORECASE
        )
        if (entity == "height" or entity == "width") and not unit:
            unit = "inch"
            sstr = ""
        elif not unit:
            continue
        else:
            unit = unit.group(0).strip()
            sstr = unit
            unit = unit.lower()
        if unit == "feet":
            unit = "foot"
        # Convert the unit to full form
        if unit in unit_short_form:
            unit = unit_short_form[unit]
        # Check if the unit is valid
        if unit in entity_unit:
            entity_value = value.replace(sstr, "").strip()
            # Replace I with 1 and O with 0
            entity_value = (
                entity_value.replace("I", "1")
                .replace("i", "1")
                .replace("J", "1")
                .replace("j", "1")
                .replace("O", "0")
                .replace("o", "0")
                .replace("D", "0")
                .replace("B", "8")
                .replace("b", "6")
                .replace("L", "1")
                .replace("l", "1")
                .replace("S", "5")
                .replace("s", "5")
                .replace("Z", "2")
                .replace("z", "2")
                .replace("Q", "0")
                .replace("q", "9")
            )
            try:
                entity_value = float(entity_value)
                if int(entity_value) == entity_value and entity_value > 1000:
                    entity_value = int(entity_value)
            except:
                continue
            return str(entity_value) + " " + unit
    for m in reverse_match:
        value = m.group(0)
        unit = re.search(
            regex_units[entity], value, re.IGNORECASE
        )
        if entity == "height" and not unit:
            unit = "inch"
            sstr = ""
        elif not unit:
            continue
        else:
            unit = unit.group(0).strip()
            sstr = unit
            unit = unit.lower()
        if unit == "feet":
            unit = "foot"
        # Convert the unit to full form
        if unit in unit_short_form:
            unit = unit_short_form[unit]
        # Check if the unit is valid
        if unit in entity_unit:
            entity_value = value.replace(sstr, "").strip()
            # Replace I with 1 and O with 0
            entity_value = (
                entity_value.replace("I", "1")
                .replace("i", "1")
                .replace("J", "1")
                .replace("j", "1")
                .replace("O", "0")
                .replace("o", "0")
                .replace("D", "0")
            )
            try:
                # Reverse the value
                entity_value = entity_value[::-1]
                entity_value = float(entity_value)
                if int(entity_value) == entity_value and entity_value > 1000:
                    entity_value = int(entity_value)
            except:
                continue
            return str(entity_value) + " " + unit
    return None

In [7]:
test_data = pd.read_csv("/content/drive/MyDrive/ML challenge/merged_output_final.csv")
test_data = test_data.dropna(subset=["OCR Text"])
test_data.head()

,index,image_link,group_id,entity_name,OCR Text
0,0,https://m.media-amazon.com/images/I/110EibNycl...,156839,height,2.63in 6.68cm 91.44cm - 199.39cm 36in - 78in
1,1,https://m.media-amazon.com/images/I/11TU2clswz...,792578,width,"Size Width Length One Size 42cm/16.54"" 200cm/7..."
2,2,https://m.media-amazon.com/images/I/11TU2clswz...,792578,height,"Size Width Length One Size 42cm/16.54"" 200cm/7..."
3,3,https://m.media-amazon.com/images/I/11TU2clswz...,792578,depth,"Size Width Length One Size 42cm/16.54"" 200cm/7..."
4,4,https://m.media-amazon.com/images/I/11gHj8dhhr...,792578,depth,"Size Width Length One Size 10.50cm/4.13"" 90cm/..."


In [13]:
print(process_text("cm 73/ 9", "height"))

None


In [8]:
test_input = pd.read_csv("/content/drive/MyDrive/ML challenge/merged_output_final.csv")

In [10]:
test_input = test_input[test_input["entity_name"] == "weight"]

In [17]:
df = pd.DataFrame(columns=["index", "prediction"], index=list(range(len(test_input))))

In [18]:
df

,index,prediction
0,NaN,NaN
1,NaN,NaN
2,NaN,NaN
3,NaN,NaN
4,NaN,NaN
...,...,...
28141,NaN,NaN
28142,NaN,NaN
28143,NaN,NaN
28144,NaN,NaN


In [20]:
for i, index in enumerate(tqdm(test_input["index"], total=len(test_input))):
    if index in test_data["index"]:
        row = test_data[test_data["index"] == index]
        if len(row) == 0:
            df.loc[i] = [index, ""]
            continue
        context = row["OCR Text"].values[0]
        entity = row["entity_name"].values[0]
        if entity == "voltage":
            answer = qa_pipeline(question=voltage_question, context=context)
        elif entity == "depth":
            answer = qa_pipeline(question=depth_question, context=context)
            print(answer["answer"])
        else:
            answer = qa_pipeline(question=question.replace("%%", entity), context=context)
        # prediction = process_text(answer["answer"], entity)
        if answer["answer"] != None:
            df.loc[i] = [index, answer["answer"]]
        else:
            df.loc[i] = [index, ""]
    else:
        df.loc[i] = [index, ""]


  0%|          | 18/28146 [00:00<05:29, 85.35it/s]

200cm/78.74"
90cm/35.43"
4.
185 cm 32cm
3.8in 9cm
5.8
30 5.4 in 90 cm 47.2 in 120cm
5.
90 cm 35 in 47 in 120 cm
5 in -12.5 cm
5.4 in 90 cm 47.2 in 120 cm
100mm- -280mm- 280mm
4.
9.1in 23cm
4.7in/12 cm 3.1in/8cm
39
14.9 in/38cm 4.33in/11cm
45cm/17.7in 10cm/3.9in


  0%|          | 36/28146 [00:00<05:30, 85.02it/s]

Biogents
25.
20.0cm 2.9 cm
100mm/3.93in
82in 208.3 cm
90cm/35.4in 10cm/3.9in
50cm
7.86 in/20cm 3.14 in/8cm
H35cm D10.5cm
0.79inch
5.9 in/15 cm
7.19cm/ 2.83inch
2.7 in/7.0 cm
40cm 61cm
37.5cm/14.76in
10 in 7.4
213.5mm 91mm 85.7mm
1.1 inch 2.7 cm


  0%|          | 45/28146 [00:00<05:26, 85.99it/s]

...

1.77inch 1.25inch 1inch
274cm/108inch
27cm 16cm
Classic Black Frame Product Prtection Product Packaging
140"210"
XFX1Ocm 03
5.4in
2.7cm
19oz 11inch 3.7inch
6M M
4cm/1.57in
25.
40CM/15.8INCH
104cm/40.9in
14'x14 35x35cm
80mm/ 3.15inch


100%|█████████▉| 28139/28146 [06:49<00:00, 71.06it/s]

42cm
20cm/7.9in
OORAO GDODE
140cm
22cm (8.
3.1in 3.5in
2POSTERSA3-29.7X42CM
Sinple 72" 15.3"
31CM
24pcs
Deuteronomy 28:6
3ft/90cm
CM
17.5cm/6.89inch
6inch 4inch


100%|██████████| 28146/28146 [06:49<00:00, 68.69it/s]

6.6in/17cm
88mm /3.46" 64mm / 2.52'
2.8CM1.1in
0.85mm
360MM 14 INCHES
27.5 in 23.6 in
9 in 8 in


In [21]:
df

,index,prediction
0,3,"200cm/78.74"""
1,4,"90cm/35.43"""
2,12,4.
3,24,185 cm 32cm
4,26,3.8in 9cm
...,...,...
28141,108390,2.8CM1.1in
28142,108393,0.85mm
28143,108394,360MM 14 INCHES
28144,108397,27.5 in 23.6 in


In [23]:
df.to_csv("DepthLongestPreProcessed.csv")

In [ ]:
df2 = df.copy()
df2.head(n=20)

,index,prediction
0,0,36.0 inch
1,1,200.0 centimetre
2,2,200.0 centimetre
3,3,200.0 centimetre
4,4,90.0 centimetre
5,5,90.0 centimetre
6,6,90.0 centimetre
7,7,3.56 centimetre
8,8,40.0 centimetre
9,9,40.0 centimetre


In [ ]:
df2.to_csv("predictions.csv", index=False)

In [ ]:
df = pd.read_csv("predictions.csv")

In [ ]:
heights = pd.read_csv("../dataset/height_yolo.csv")
for row in tqdm(heights.iterrows(), total=len(heights)):
    row = row[1]
    index = row["index"]
    text = str(row["OCR Text"])
    if text == "LLM maybe":
        continue
    answer = process_text(text, "height")
    if answer:
        df.loc[df["index"] == index] = [index, answer]

100%|██████████| 32282/32282 [00:14<00:00, 2233.62it/s]


In [ ]:
widths = pd.read_csv("../dataset/width_text.csv")
for row in tqdm(widths.iterrows(), total=len(widths)):
    row = row[1]
    index = row["index"]
    text = str(row["OCR Text"])
    if text == "LLM maybe":
        continue
    answer = process_text(text, "width")
    if answer:
        df.loc[df["index"] == index] = [index, answer]


100%|██████████| 26931/26931 [00:12<00:00, 2182.40it/s]


In [ ]:
widths_vllm = pd.read_csv("../dataset/width_VLM.csv")
for row in tqdm(widths_vllm.iterrows(), total=len(widths_vllm)):
    row = row[1]
    index = row["index"]
    entity = row["entity_name"]
    text = str(row["prediction"])
    if text == "LLM maybe":
        text = str(row["OCR Text"])
    answer = qa_pipeline(question=question.replace("%%", entity), context=text)
    prediction = process_text(answer["answer"], entity)
    if prediction:
        df.loc[df["index"] == index] = [index, prediction]

100%|██████████| 453/453 [00:06<00:00, 66.26it/s]


In [ ]:
widths_vllm = pd.read_csv("../dataset/width_VLM1.csv")
for row in tqdm(widths_vllm.iterrows(), total=len(widths_vllm)):
    row = row[1]
    index = row["index"]
    entity = row["entity_name"]
    text = str(row["prediction"])
    if text == "LLM maybe":
        text = str(row["OCR Text"])
    answer = qa_pipeline(question=question.replace("%%", entity), context=text)
    prediction = process_text(answer["answer"], entity)
    if prediction:
        df.loc[df["index"] == index] = [index, prediction]

100%|██████████| 1080/1080 [00:14<00:00, 73.95it/s]


In [ ]:
heights_vllm = pd.read_csv("../dataset/height_VLM.csv")
for row in tqdm(heights_vllm.iterrows(), total=len(heights_vllm)):
    row = row[1]
    index = row["index"]
    entity = row["entity_name"]
    text = str(row["prediction"])
    if text == "LLM maybe":
        text = str(row["OCR Text"])
    answer = qa_pipeline(question=question.replace("%%", entity), context=text)
    prediction = process_text(answer["answer"], entity)
    if prediction:
        df.loc[df["index"] == index] = [index, prediction]

100%|██████████| 734/734 [00:10<00:00, 67.34it/s]


In [ ]:
df.to_csv(f"predictions{time()}.csv", index=False)

In [ ]:
!python sanity.py --test_filename ../dataset/test.csv --output_filename predictions1726421634.3328924.csv

Parsing successfull for file: predictions1726421634.3328924.csv


In [ ]:
edge_case = [
    "The height of the object in the image is 9 inches."
    "139.5cm",
]

for case in edge_case:
    answer = process_text(case, "height")
    print(answer)

9.0 inch


In [ ]:
def process_text(answer, entity):
    # Clean up the output of the LLM
    values_list = []
    entity_unit_map = {
        "width": {"centimetre", "foot", "millimetre", "metre", "inch", "yard"},
        "depth": {"centimetre", "foot", "millimetre", "metre", "inch", "yard"},
        "height": {"centimetre", "foot", "millimetre", "metre", "inch", "yard"},
        "item_weight": {
            "milligram",
            "kilogram",
            "microgram",
            "gram",
            "ounce",
            "ton",
            "pound",
        },
        "voltage": {"millivolt", "kilovolt", "volt"},
        "wattage": {"kilowatt", "watt"},
        "item_volume": {
            "cubic foot",
            "microlitre",
            "cup",
            "fluid ounce",
            "centilitre",
            "imperial gallon",
            "pint",
            "decilitre",
            "litre",
            "millilitre",
            "quart",
            "cubic inch",
            "gallon",
            "ounce",
        },
        "maximum_weight_recommendation": {
            "milligram",
            "kilogram",
            "microgram",
            "gram",
            "ounce",
            "ton",
            "pound",
        },
    }
    unit_short_form = {
        "cm": "centimetre",
        "mm": "millimetre",
        "m": "metre",
        "in": "inch",
        "ft": "foot",
        "yd": "yard",
        "g": "gram",
        "kg": "kilogram",
        "mg": "milligram",
        "lb": "pound",
        "oz": "ounce",
        "ton": "ton",
        "ug": "microgram",
        "lbs": "pound",
        "9": "gram",
        "ozs": "ounce",
        "mv": "millivolt",
        "kv": "kilovolt",
        "v": "volt",
        "kw": "kilowatt",
        "w": "watt",
        "cf": "cubic foot",
        "ul": "microlitre",
        "fl oz": "fluid ounce",
        "cl": "centilitre",
        "gal": "imperial gallon",
        "pt": "pint",
        "dl": "decilitre",
        "l": "litre",
        "ml": "millilitre",
        "qt": "quart",
        "cu in": "cubic inch",
        "gals": "gallon",
        "c in": "cubic inch",
        "cu ft": "cubic foot",
        "o2": "ounce",
        "0z": "ounce",
        "k9": "kilogram",
        "\'\'": "inch",
        "``": "inch",
        "\"": "inch",
        # If no unit is mentioned, assume it is inches
        "": "inch",
        "c": "centimetre",
    }
    # Replace common OCR errors
    answer = answer.replace(",", ".")
    if entity == "depth":
        entity_unit = entity_unit_map[entity]
        regex_map = {
            "item_weight": r"[0-9IJOQSZDLB]+(\.[0-9IJOQSZDLB]+)?\s*(g|kg|mg|ug|oz|ton|lb|lbs|Ibs|bs|1bs|ozs|o2|0z|k9|kilo|milli|micro|ounce|tonne|pound|gram)",
            "width": r"[0-9IJOQSZDLB]+(\.[0-9IJOQSZDLB]+)?\s*(c|cm|mm|m|in|ft|yd|centi|milli|inch|foot|yard|\'\'|``|\"|feet)",
            "depth": r"[0-9IJOQSZDLB]+(\.[0-9IJOQSZDLB]+)?\s*(c|cm|mm|m|in|ft|yd|centi|milli|inch|foot|yard|\'\'|``|\"|feet)",
            "height": r"[0-9IJOQSZDLB]+(\.[0-9IJOQSZDLB]+)?\s*(c|cm|mm|m|in|ft|yd|centi|milli|inch|foot|yard|\'\'|``|\"|feet)",
            "voltage": r"[0-9IJOQSZDLB]+(\.[0-9IJOQSZDLB]+)?\s*(mv|kv|v|milli|kilo)",
            "wattage": r"[0-9IJOQSZDLB]+(\.[0-9IJOQSZDLB]+)?\s*(kw|w|kilo)",
            "item_volume": r"[0-9IJOQSZDLB]+(\.[0-9IJOQSZDLB]+)?\s*(cf|ul|fl oz|cl|gal|pt|dl|l|ml|qt|cu in|gals|c in|cu ft|o2|oz|ozs|0z|cubic|micro|cup|fluid|centi|imperial|pint|decilitre|litre|millilitre|quart|gallon|ounce)",
            "maximum_weight_recommendation": r"[0-9IJOQSZDLB]+(\.[0-9IJOQSZDLB]+)?\s*(g|kg|mg|ug|oz|ton|lb|lbs|Ibs|bs|1bs|ozs|o2|02|0z|k9|kilo|milli|micro|ounce|tonne|pound|gram)",
        }
        reverse_regex_map = {
            "item_weight": r"\s*(g|kg|mg|ug|oz|ton|lb|lbs|Ibs|bs|1bs|ozs|o2|0z|k9|kilo|milli|micro|ounce|tonne|pound|gram)\s*[0-9IJOQSZDLB]+(\.[0-9IJOQSZDLB]+)?",
            "width": r"\s*(c|cm|mm|m|in|ft|yd|centi|milli|inch|foot|yard|\'\'|``|\"|feet)\s*[0-9IJOQSZDLB]+(\.[0-9IJOQSZDLB]+)?",
            "depth": r"\s*(c|cm|mm|m|in|ft|yd|centi|milli|inch|foot|yard|\'\'|``|\"|feet)\s*[0-9IJOQSZDLB]+(\.[0-9IJOQSZDLB]+)?",
            "height": r"\s*(c|cm|mm|m|in|ft|yd|centi|milli|inch|foot|yard|\'\'|``|\"|feet)\s*[0-9IJOQSZDLB]+(\.[0-9IJOQSZDLB]+)?",
            "voltage": r"\s*(mv|kv|v|milli|kilo)\s*[0-9IJOQSZDLB]+(\.[0-9IJOQSZDLB]+)?",
            "wattage": r"\s*(kw|w|kilo)\s*[0-9IJOQSZDLB]+(\.[0-9IJOQSZDLB]+)?",
            "item_volume": r"\s*(cf|ul|fl oz|cl|gal|pt|dl|l|ml|qt|cu in|gals|c in|cu ft|o2|oz|ozs|0z|cubic|micro|cup|fluid|centi|imperial|pint|decilitre|litre|millilitre|quart|gallon|ounce)\s*[0-9IJOQSZDLB]+(\.[0-9IJOQSZDLB]+)?",
            "maximum_weight_recommendation": r"\s*(g|kg|mg|ug|oz|ton|lb|lbs|Ibs|bs|1bs|ozs|o2|02|0z|k9|kilo|milli|micro|ounce|tonne|pound|gram)\s*[0-9IJOQSZDLB]+(\.[0-9IJOQSZDLB]+)?",
        }
        regex_units = {
            "item_weight": r"\s*(g|kg|mg|ug|oz|ton|lb|lbs|Ibs|bs|1bs|ozs|o2|0z|k9|kilo|milli|micro|ounce|tonne|pound|gram)",
            "width": r"\s*(c|cm|mm|m|in|ft|yd|centi|milli|inch|foot|yard|\'\'|``|\"|feet)",
            "depth": r"\s*(c|cm|mm|m|in|ft|yd|centi|milli|inch|foot|yard|\'\'|``|\"|feet)",
            "height": r"\s*(c|cm|mm|m|in|ft|yd|centi|milli|inch|foot|yard|\'\'|``|\"|feet)",
            "voltage": r"\s*(mv|kv|v|milli|kilo)",
            "wattage": r"\s*(kw|w|kilo)",
            "item_volume": r"\s*(cf|ul|fl oz|cl|gal|pt|dl|l|ml|qt|cu in|gals|c in|cu ft|o2|oz|ozs|0z|cubic|micro|cup|fluid|centi|imperial|pint|decilitre|litre|millilitre|quart|gallon|ounce)",
            "maximum_weight_recommendation": r"\s*(g|kg|mg|ug|oz|ton|lb|lbs|Ibs|bs|1bs|ozs|o2|0z|k9|kilo|milli|micro|ounce|tonne|pound|gram)",
        }
        # Remove igh which will cause false positives
        answer = answer.replace("igh", "")
        # Remove ima which will cause false positives
        answer = answer.replace("ima", "")
        # Replace comma with dot
        answer = answer.replace(",", ".")
        # Match the regex ignoring case
        match = re.finditer(regex_map[entity], answer, re.IGNORECASE)
        reverse_match = re.finditer(reverse_regex_map[entity], answer, re.IGNORECASE)
        for m in match:
            # Extract the number and the unit
            # Extract the matched value
            value = m.group(0)
            # Extract the unit
            unit = re.search(
                regex_units[entity], value, re.IGNORECASE
            )
            if (entity == "height" or entity == "width") and not unit:
                unit = "inch"
                sstr = ""
            elif not unit:
                continue
            else:
                unit = unit.group(0).strip()
                sstr = unit
                unit = unit.lower()
            if unit == "feet":
                unit = "foot"
            # Convert the unit to full form
            if unit in unit_short_form:
                unit = unit_short_form[unit]
            # Check if the unit is valid
            if unit in entity_unit:
                entity_value = value[: value.index(sstr)].strip()
                # Replace I with 1 and O with 0
                entity_value = (
                    entity_value.replace("I", "1")
                    .replace("i", "1")
                    .replace("J", "1")
                    .replace("j", "1")
                    .replace("O", "0")
                    .replace("o", "0")
                    .replace("D", "0")
                    .replace("B", "8")
                    .replace("b", "6")
                    .replace("L", "1")
                    .replace("l", "1")
                    .replace("S", "5")
                    .replace("s", "5")
                    .replace("Z", "2")
                    .replace("z", "2")
                    .replace("Q", "0")
                    .replace("q", "9")
                )
                try:
                    entity_value = float(entity_value)
                    if int(entity_value) == entity_value and entity_value > 1000:
                        entity_value = int(entity_value)
                except:
                    continue
                values_list.append((entity_value, unit))
        for m in reverse_match:
            value = m.group(0)
            unit = re.search(
                regex_units[entity], value, re.IGNORECASE
            )
            if entity == "height" and not unit:
                unit = "inch"
                sstr = ""
            elif not unit:
                continue
            else:
                unit = unit.group(0).strip()
                sstr = unit
                unit = unit.lower()
            if unit == "feet":
                unit = "foot"
            # Convert the unit to full form
            if unit in unit_short_form:
                unit = unit_short_form[unit]
            # Check if the unit is valid
            if unit in entity_unit:
                entity_value = value.replace(sstr, "").strip()
                # Replace I with 1 and O with 0
                entity_value = (
                    entity_value.replace("I", "1")
                    .replace("i", "1")
                    .replace("J", "1")
                    .replace("j", "1")
                    .replace("O", "0")
                    .replace("o", "0")
                    .replace("D", "0")
                    .replace("B", "8")
                    .replace("b", "6")
                    .replace("L", "1")
                    .replace("l", "1")
                    .replace("S", "5")
                    .replace("s", "5")
                    .replace("Z", "2")
                    .replace("z", "2")
                    .replace("Q", "0")
                    .replace("q", "9")
                )
                try:
                    # Reverse the value
                    entity_value = entity_value[::-1]
                    entity_value = float(entity_value)
                    if int(entity_value) == entity_value and entity_value > 1000:
                        entity_value = int(entity_value)
                except:
                    continue
                values_list.append((entity_value, unit))
          max_tuple = max(values_list, key=lambda x: x[0])
          return str(max_tuple[0]) + " " + max_tuple[1]
    if entity == "voltage":
    # Answer should be of the form [x, y] volt
    # Extract the voltage range
        pattern = r"[0-9IJOQSZDLBLBLBLBLBLBLB]+(\.[0-9IJOQSZDLB]+)?[-/][0-9IJOQSZDLB]+(\.[0-9IJOQSZDLB]+)?\s*(mv|kv|v|milli|kilo)|[0-9IJOQSZDLB]+\s*(mv|kv|v|milli|kilo)"
        voltage_ranges = re.finditer(pattern, answer, re.IGNORECASE)
        if not voltage_ranges:
            return None
        for vr in voltage_ranges:
            voltage_range = vr.group(0)
            # Extract the unit
            unit = re.search(
                r"\s*(mv|kv|v|milli|kilo)", voltage_range, re.IGNORECASE
            )
            if not unit:
                continue
            unit = unit.group(0).strip()
            voltage_range = voltage_range.replace(unit, "").strip()
            voltage_range = (
                voltage_range.replace("I", "1")
                .replace("i", "1")
                .replace("J", "1")
                .replace("j", "1")
                .replace("O", "0")
                .replace("o", "0")
                .replace("D", "0")
            )
            # Convert the unit to full form
            unit = unit.lower()
            if unit in unit_short_form:
                unit = unit_short_form[unit]
            # Check if the unit is valid
            if unit in entity_unit_map[entity]:
                if "-" in voltage_range or "/" in voltage_range:
                    voltage_range = voltage_range.split("-") if "-" in voltage_range else voltage_range.split("/")
                    voltage_range = float(voltage_range[0])
                    return str(voltage_range) + " " + unit




    entity_unit = entity_unit_map[entity]
    regex_map = {
        "item_weight": r"[0-9IJOQSZDLB]+(\.[0-9IJOQSZDLB]+)?\s*(g|kg|mg|ug|oz|ton|lb|lbs|Ibs|bs|1bs|ozs|o2|0z|k9|kilo|milli|micro|ounce|tonne|pound|gram)",
        "width": r"[0-9IJOQSZDLB]+(\.[0-9IJOQSZDLB]+)?\s*(c|cm|mm|m|in|ft|yd|centi|milli|inch|foot|yard|\'\'|``|\"|feet)",
        "depth": r"[0-9IJOQSZDLB]+(\.[0-9IJOQSZDLB]+)?\s*(c|cm|mm|m|in|ft|yd|centi|milli|inch|foot|yard|\'\'|``|\"|feet)",
        "height": r"[0-9IJOQSZDLB]+(\.[0-9IJOQSZDLB]+)?\s*(c|cm|mm|m|in|ft|yd|centi|milli|inch|foot|yard|\'\'|``|\"|feet)",
        "voltage": r"[0-9IJOQSZDLB]+(\.[0-9IJOQSZDLB]+)?\s*(mv|kv|v|milli|kilo)",
        "wattage": r"[0-9IJOQSZDLB]+(\.[0-9IJOQSZDLB]+)?\s*(kw|w|kilo)",
        "item_volume": r"[0-9IJOQSZDLB]+(\.[0-9IJOQSZDLB]+)?\s*(cf|ul|fl oz|cl|gal|pt|dl|l|ml|qt|cu in|gals|c in|cu ft|o2|oz|ozs|0z|cubic|micro|cup|fluid|centi|imperial|pint|decilitre|litre|millilitre|quart|gallon|ounce)",
        "maximum_weight_recommendation": r"[0-9IJOQSZDLB]+(\.[0-9IJOQSZDLB]+)?\s*(g|kg|mg|ug|oz|ton|lb|lbs|Ibs|bs|1bs|ozs|o2|02|0z|k9|kilo|milli|micro|ounce|tonne|pound|gram)",
    }
    reverse_regex_map = {
        "item_weight": r"\s*(g|kg|mg|ug|oz|ton|lb|lbs|Ibs|bs|1bs|ozs|o2|0z|k9|kilo|milli|micro|ounce|tonne|pound|gram)\s*[0-9IJOQSZDLB]+(\.[0-9IJOQSZDLB]+)?",
        "width": r"\s*(c|cm|mm|m|in|ft|yd|centi|milli|inch|foot|yard|\'\'|``|\"|feet)\s*[0-9IJOQSZDLB]+(\.[0-9IJOQSZDLB]+)?",
        "depth": r"\s*(c|cm|mm|m|in|ft|yd|centi|milli|inch|foot|yard|\'\'|``|\"|feet)\s*[0-9IJOQSZDLB]+(\.[0-9IJOQSZDLB]+)?",
        "height": r"\s*(c|cm|mm|m|in|ft|yd|centi|milli|inch|foot|yard|\'\'|``|\"|feet)\s*[0-9IJOQSZDLB]+(\.[0-9IJOQSZDLB]+)?",
        "voltage": r"\s*(mv|kv|v|milli|kilo)\s*[0-9IJOQSZDLB]+(\.[0-9IJOQSZDLB]+)?",
        "wattage": r"\s*(kw|w|kilo)\s*[0-9IJOQSZDLB]+(\.[0-9IJOQSZDLB]+)?",
        "item_volume": r"\s*(cf|ul|fl oz|cl|gal|pt|dl|l|ml|qt|cu in|gals|c in|cu ft|o2|oz|ozs|0z|cubic|micro|cup|fluid|centi|imperial|pint|decilitre|litre|millilitre|quart|gallon|ounce)\s*[0-9IJOQSZDLB]+(\.[0-9IJOQSZDLB]+)?",
        "maximum_weight_recommendation": r"\s*(g|kg|mg|ug|oz|ton|lb|lbs|Ibs|bs|1bs|ozs|o2|02|0z|k9|kilo|milli|micro|ounce|tonne|pound|gram)\s*[0-9IJOQSZDLB]+(\.[0-9IJOQSZDLB]+)?",
    }
    regex_units = {
        "item_weight": r"\s*(g|kg|mg|ug|oz|ton|lb|lbs|Ibs|bs|1bs|ozs|o2|0z|k9|kilo|milli|micro|ounce|tonne|pound|gram)",
        "width": r"\s*(c|cm|mm|m|in|ft|yd|centi|milli|inch|foot|yard|\'\'|``|\"|feet)",
        "depth": r"\s*(c|cm|mm|m|in|ft|yd|centi|milli|inch|foot|yard|\'\'|``|\"|feet)",
        "height": r"\s*(c|cm|mm|m|in|ft|yd|centi|milli|inch|foot|yard|\'\'|``|\"|feet)",
        "voltage": r"\s*(mv|kv|v|milli|kilo)",
        "wattage": r"\s*(kw|w|kilo)",
        "item_volume": r"\s*(cf|ul|fl oz|cl|gal|pt|dl|l|ml|qt|cu in|gals|c in|cu ft|o2|oz|ozs|0z|cubic|micro|cup|fluid|centi|imperial|pint|decilitre|litre|millilitre|quart|gallon|ounce)",
        "maximum_weight_recommendation": r"\s*(g|kg|mg|ug|oz|ton|lb|lbs|Ibs|bs|1bs|ozs|o2|0z|k9|kilo|milli|micro|ounce|tonne|pound|gram)",
    }
    # Remove igh which will cause false positives
    answer = answer.replace("igh", "")
    # Remove ima which will cause false positives
    answer = answer.replace("ima", "")
    # Replace comma with dot
    answer = answer.replace(",", ".")
    # Match the regex ignoring case
    match = re.finditer(regex_map[entity], answer, re.IGNORECASE)
    reverse_match = re.finditer(reverse_regex_map[entity], answer, re.IGNORECASE)

    # Process each match
    for m in match:
        # Extract the number and the unit
        # Extract the matched value
        value = m.group(0)
        # Extract the unit
        unit = re.search(
            regex_units[entity], value, re.IGNORECASE
        )
        if (entity == "height" or entity == "width") and not unit:
            unit = "inch"
            sstr = ""
        elif not unit:
            continue
        else:
            unit = unit.group(0).strip()
            sstr = unit
            unit = unit.lower()
        if unit == "feet":
            unit = "foot"
        # Convert the unit to full form
        if unit in unit_short_form:
            unit = unit_short_form[unit]
        # Check if the unit is valid
        if unit in entity_unit:
            entity_value = value[: value.index(sstr)].strip()
            # Replace I with 1 and O with 0
            entity_value = (
                entity_value.replace("I", "1")
                .replace("i", "1")
                .replace("J", "1")
                .replace("j", "1")
                .replace("O", "0")
                .replace("o", "0")
                .replace("D", "0")
                .replace("B", "8")
                .replace("b", "6")
                .replace("L", "1")
                .replace("l", "1")
                .replace("S", "5")
                .replace("s", "5")
                .replace("Z", "2")
                .replace("z", "2")
                .replace("Q", "0")
                .replace("q", "9")
            )
            try:
                entity_value = float(entity_value)
                if int(entity_value) == entity_value and entity_value > 1000:
                    entity_value = int(entity_value)
            except:
                continue
          return str(entity_value) + " " + unit
    for m in reverse_match:
        value = m.group(0)
        unit = re.search(
            regex_units[entity], value, re.IGNORECASE
        )
        if entity == "height" and not unit:
            unit = "inch"
            sstr = ""
        elif not unit:
            continue
        else:
            unit = unit.group(0).strip()
            sstr = unit
            unit = unit.lower()
        if unit == "feet":
            unit = "foot"
        # Convert the unit to full form
        if unit in unit_short_form:
            unit = unit_short_form[unit]
        # Check if the unit is valid
        if unit in entity_unit:
            entity_value = value.replace(sstr, "").strip()
            # Replace I with 1 and O with 0
            entity_value = (
                entity_value.replace("I", "1")
                .replace("i", "1")
                .replace("J", "1")
                .replace("j", "1")
                .replace("O", "0")
                .replace("o", "0")
                .replace("D", "0")
                .replace("B", "8")
                .replace("b", "6")
                .replace("L", "1")
                .replace("l", "1")
                .replace("S", "5")
                .replace("s", "5")
                .replace("Z", "2")
                .replace("z", "2")
                .replace("Q", "0")
                .replace("q", "9")
            )
            try:
                # Reverse the value
                entity_value = entity_value[::-1]
                entity_value = float(entity_value)
                if int(entity_value) == entity_value and entity_value > 1000:
                    entity_value = int(entity_value)
            except:
                continue
            return str(entity_value) + " " + unit
    return None

In [24]:
def process_text(answer, entity):
    # Clean up the output of the LLM
    entity_unit_map = {
        "depth": {"centimetre", "foot", "millimetre", "metre", "inch", "yard"}
    }
    unit_short_form = {
        "cm": "centimetre",
        "mm": "millimetre",
        "m": "metre",
        "in": "inch",
        "ft": "foot",
        "yd": "yard",
        "g": "gram",
        "kg": "kilogram",
        "mg": "milligram",
        "lb": "pound",
        "oz": "ounce",
        "ton": "ton",
        "ug": "microgram",
        "lbs": "pound",
        "9": "gram",
        "ozs": "ounce",
        "mv": "millivolt",
        "kv": "kilovolt",
        "v": "volt",
        "kw": "kilowatt",
        "w": "watt",
        "cf": "cubic foot",
        "ul": "microlitre",
        "fl oz": "fluid ounce",
        "cl": "centilitre",
        "gal": "imperial gallon",
        "pt": "pint",
        "dl": "decilitre",
        "l": "litre",
        "ml": "millilitre",
        "qt": "quart",
        "cu in": "cubic inch",
        "gals": "gallon",
        "c in": "cubic inch",
        "cu ft": "cubic foot",
        "o2": "ounce",
        "0z": "ounce",
        "k9": "kilogram",
        "\'\'": "inch",
        "``": "inch",
        "\"": "inch",
        # If no unit is mentioned, assume it is inches
        "": "inch",
        "c": "centimetre",
    }
    # Replace common OCR errors
    answer = answer.replace(",", ".")


    entity_unit = entity_unit_map[entity]
    regex_map = {
        "depth": r"[0-9IJOQSZDLB]+(\.[0-9IJOQSZDLB]+)?\s*(c|cm|mm|m|in|ft|yd|centi|milli|inch|foot|yard|\'\'|``|\"|feet)"
    }
    reverse_regex_map = {
        "depth": r"\s*(c|cm|mm|m|in|ft|yd|centi|milli|inch|foot|yard|\'\'|``|\"|feet)\s*[0-9IJOQSZDLB]+(\.[0-9IJOQSZDLB]+)?"
    }
    regex_units = {
        "depth": r"\s*(c|cm|mm|m|in|ft|yd|centi|milli|inch|foot|yard|\'\'|``|\"|feet)"
    }
    # Remove igh which will cause false positives
    answer = answer.replace("igh", "")
    # Remove ima which will cause false positives
    answer = answer.replace("ima", "")
    # Replace comma with dot
    answer = answer.replace(",", ".")
    # Match the regex ignoring case
    match = re.finditer(regex_map[entity], answer, re.IGNORECASE)
    reverse_match = re.finditer(reverse_regex_map[entity], answer, re.IGNORECASE)
    length_list = []
    for m in match:
      print("In forward pass:")
      # Extract the number and the unit
      # Extract the matched value
      value = m.group(0)
      # Extract the unit
      unit = re.search(
          regex_units[entity], value, re.IGNORECASE
      )
      if not unit:
          unit = "inch"
          sstr = ""
      else:
          unit = unit.group(0).strip()
          sstr = unit
          unit = unit.lower()
      if unit == "feet":
          unit = "foot"
      # Convert the unit to full form
      if unit in unit_short_form:
          unit = unit_short_form[unit]
      # Check if the unit is valid
      if unit in entity_unit:
          entity_value = value[: value.index(sstr)].strip()
          # Replace I with 1 and O with 0
          entity_value = (
              entity_value.replace("I", "1")
              .replace("i", "1")
              .replace("J", "1")
              .replace("j", "1")
              .replace("O", "0")
              .replace("o", "0")
              .replace("D", "0")
              .replace("B", "8")
              .replace("b", "6")
              .replace("L", "1")
              .replace("l", "1")
              .replace("S", "5")
              .replace("s", "5")
              .replace("Z", "2")
              .replace("z", "2")
              .replace("Q", "0")
              .replace("q", "9")
          )
          try:
              entity_value = float(entity_value)
              length_list.append((entity_value, unit))
          except:
              continue
    for m in reverse_match:
        value = m.group(0)
        unit = re.search(
            regex_units[entity], value, re.IGNORECASE
        )
        if not unit:
            unit = "inch"
            sstr = ""
        else:
            unit = unit.group(0).strip()
            sstr = unit
            unit = unit.lower()
        if unit == "feet":
            unit = "foot"
        # Convert the unit to full form
        if unit in unit_short_form:
            unit = unit_short_form[unit]
        # Check if the unit is valid
        if unit in entity_unit:
            entity_value = value.replace(sstr, "").strip()
          # Replace I with 1 and O with 0
        entity_value = (
            entity_value.replace("I", "1")
            .replace("i", "1")
            .replace("J", "1")
            .replace("j", "1")
            .replace("O", "0")
            .replace("o", "0")
            .replace("D", "0")
            .replace("B", "8")
            .replace("b", "6")
            .replace("L", "1")
            .replace("l", "1")
            .replace("S", "5")
            .replace("s", "5")
            .replace("Z", "2")
            .replace("z", "2")
            .replace("Q", "0")
            .replace("q", "9")
        )
        try:
            # Reverse the value
            entity_value = entity_value[::-1]
            entity_value = float(entity_value)
            entity_value = float(entity_value)
            length_list.append((entity_value, unit))
        except:
            continue
    if len(lengths) != 0:
      max_measurement = max(length_list, key=lambda x: x[0])
      max_len = max_measurement[0]
      max_unit = max_measurement[1]
      if int(max_len) == max_len and max_len > 1000:
                    max_len = int(max_len)
      return str(max_len) + " " + max_unit
    else:
      return None



In [28]:
print("4", "depth")

4 depth


In [52]:
import re

def process_text(answer, entity):
    # Clean up the output of the LLM
    entity_unit_map = {
        "depth": {"cm", "ft", "mm", "m", "in", "yd"}
    }
    unit_standard_form = {
        "cm": "cm",
        "mm": "mm",
        "m": "m",
        "in": "in",
        "ft": "ft",
        "yd": "yd",
        "c": "cm",
        "centi": "cm",
        "milli": "mm",
        "inch": "in",
        "foot": "ft",
        "yard": "yd",
        "''": "in",
        "``": "in",
        "\"": "in",
        # Add other mappings as needed
    }

    # Map standard units to full forms
    unit_full_form = {
        "cm": "centimeter",
        "mm": "millimeter",
        "m": "meter",
        "in": "inch",
        "ft": "foot",
        "yd": "yard",
    }

    # Replace common OCR errors and problematic substrings
    answer = answer.replace(",", ".")
    answer = answer.replace("igh", "")
    answer = answer.replace("ima", "")
    # Add spaces around '/' to handle measurements separated by '/'
    answer = answer.replace('/', ' / ')

    entity_unit = entity_unit_map[entity]
    regex_map = {
        "depth": r"[0-9IJOQSZDLB\.]+(\.[0-9IJOQSZDLB\.]+)?\s*(c|cm|mm|m|in|ft|yd|centi|milli|inch|foot|yard|''|``|\"|feet)?"
    }
    reverse_regex_map = {
        "depth": r"(c|cm|mm|m|in|ft|yd|centi|milli|inch|foot|yard|''|``|\"|feet)\s*[0-9IJOQSZDLB\.]+(\.[0-9IJOQSZDLB\.]+)?"
    }
    regex_units = {
        "depth": r"(c|cm|mm|m|in|ft|yd|centi|milli|inch|foot|yard|''|``|\"|feet)"
    }

    # Match the regex ignoring case
    match = re.finditer(regex_map[entity], answer, re.IGNORECASE)
    reverse_match = re.finditer(reverse_regex_map[entity], answer, re.IGNORECASE)
    length_list = []

    # Forward matches
    for m in match:
        value = m.group(0)
        unit_match = re.search(regex_units[entity], value, re.IGNORECASE)
        if not unit_match:
            unit = "in"  # Default unit
            sstr = ""
        else:
            sstr = unit_match.group(0).strip()
            unit = sstr.lower()
            if unit == "feet":
                unit = "ft"
            if unit in unit_standard_form:
                unit = unit_standard_form[unit]
        if unit in entity_unit:
            if sstr == "":
                entity_value = value.strip()
            else:
                # Ensure sstr is in value before trying to find its index
                index = value.find(sstr)
                if index != -1:
                    entity_value = value[:index].strip()
                else:
                    # Handle the case where sstr is not in value
                    entity_value = value.strip()
            # Replace OCR error letters with digits
            entity_value = (
                entity_value.replace("I", "1")
                .replace("i", "1")
                .replace("J", "1")
                .replace("j", "1")
                .replace("O", "0")
                .replace("o", "0")
                .replace("D", "0")
                .replace("B", "8")
                .replace("b", "6")
                .replace("L", "1")
                .replace("l", "1")
                .replace("S", "5")
                .replace("s", "5")
                .replace("Z", "2")
                .replace("z", "2")
                .replace("Q", "0")
                .replace("q", "9")
            )
            try:
                entity_value_float = float(entity_value)
                length_list.append((entity_value_float, unit))
            except:
                continue

    # Reverse matches
    for m in reverse_match:
        value = m.group(0)
        unit_match = re.search(regex_units[entity], value, re.IGNORECASE)
        if not unit_match:
            unit = "in"  # Default unit
            sstr = ""
        else:
            sstr = unit_match.group(0).strip()
            unit = sstr.lower()
            if unit == "feet":
                unit = "ft"
            if unit in unit_standard_form:
                unit = unit_standard_form[unit]
        if unit in entity_unit:
            if sstr == "":
                entity_value = value.strip()
            else:
                # Remove the unit from the value
                entity_value = value.replace(sstr, "").strip()
            # Replace OCR error letters with digits
            entity_value = (
                entity_value.replace("I", "1")
                .replace("i", "1")
                .replace("J", "1")
                .replace("j", "1")
                .replace("O", "0")
                .replace("o", "0")
                .replace("D", "0")
                .replace("B", "8")
                .replace("b", "6")
                .replace("L", "1")
                .replace("l", "1")
                .replace("S", "5")
                .replace("s", "5")
                .replace("Z", "2")
                .replace("z", "2")
                .replace("Q", "0")
                .replace("q", "9")
            )
            try:
                entity_value_float = float(entity_value)
                length_list.append((entity_value_float, unit))
            except:
                continue

    if len(length_list) != 0:
        # Find the maximum numerical value without converting units
        max_measurement = max(length_list, key=lambda x: x[0])
        max_len = max_measurement[0]
        max_unit = max_measurement[1]
        # Map the unit to its full form
        full_unit = unit_full_form.get(max_unit, max_unit)
        # Format the value based on the requirements
        if max_len < 1000:
            formatted_value = f"{max_len:.1f}"
        else:
            if max_len == int(max_len):
                formatted_value = f"{int(max_len)}"
            else:
                formatted_value = f"{max_len}"
        return f"{formatted_value} {full_unit}"
    else:
        return None


In [53]:
print((process_text("185 cm 32cm", "depth")))

185.0 centimeter


In [54]:
df

,index,prediction,Result
0,3,"200cm/78.74""",200.0 cm
1,4,"90cm/35.43""",90.0 cm
2,12,4.,4.0 in
3,24,185 cm 32cm,185.0 cm
4,26,3.8in 9cm,9.0 cm
...,...,...,...
28141,108390,2.8CM1.1in,2.8 cm
28142,108393,0.85mm,0.8 mm
28143,108394,360MM 14 INCHES,360.0 mm
28144,108397,27.5 in 23.6 in,27.5 in


In [55]:
df['Result'] = None

In [56]:
for index, row in df.iterrows():
  df.at[index, 'Result'] = process_text(row['prediction'], "depth")

In [57]:
df.to_csv('depth_max.csv')

In [58]:
df2 = pd.read_csv("/content/drive/MyDrive/ML challenge/sub7.csv")

In [59]:
df2

,index,prediction
0,0,2.65 metre
1,1,42.0 centimetre
2,2,200.0 centimetre
3,3,200.0 centimetre
4,4,90.0 centimetre
...,...,...
131182,131283,NaN
131183,131284,NaN
131184,131285,NaN
131185,131286,NaN


In [60]:
# Step 1: Ensure 'index' columns are integers
df['index'] = df['index'].astype(int)
df2['index'] = df2['index'].astype(int)

# Step 2: Merge df2 with df on 'index'
merged_df = df2.merge(df[['index', 'Result']], on='index', how='left')

# Step 3: Update 'prediction' column in df2 with 'processed_result' from merged_df
# Save original predictions
df2['original_prediction'] = df2['prediction']

# Update 'prediction' with 'processed_result'
df2['prediction'] = merged_df['Result']

# Step 4: Handle missing values by filling with original predictions
df2['prediction'] = df2['prediction'].fillna(df2['original_prediction'])

# Drop the temporary 'original_prediction' column
df2.drop(columns=['original_prediction'], inplace=True)

# Display the updated df2
print(df2)


         index        prediction
0            0        2.65 metre
1            1   42.0 centimetre
2            2  200.0 centimetre
3            3  200.0 centimeter
4            4   90.0 centimeter
...        ...               ...
131182  131283               NaN
131183  131284               NaN
131184  131285               NaN
131185  131286               NaN
131186  131287               NaN

[131187 rows x 2 columns]


In [61]:
df2.to_csv("/content/drive/MyDrive/ML challenge/checkkarooo.csv")